https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -qqq transformers

     |████████████████████████████████| 2.8 MB 4.3 MB/s 
     |████████████████████████████████| 636 kB 49.2 MB/s 
     |████████████████████████████████| 3.3 MB 19.8 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 36.8 MB/s 


In [3]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('You got a GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
You got a GPU: Tesla K80


In [4]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [5]:
import re
with open('/content/drive/MyDrive/datasets/Magregor_I._Imperium_Cheloveka_Yera_K.txt', encoding='cp1251') as f:
    text = f.read()

text = re.sub('\n{2,}', '\n', text)
print(text[:100])

Империум Человека. Эра Косматики
Иван Магрегор
Степан Андреевич Зотов
Наша книга посвящена Мечте Отц


In [6]:
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

181070


array([27086, 31792,   748, 44609,    18,   625,   287, 10311,   429,
        3847])

In [7]:
tokens = np.array(tokens)
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

181070 144852 36213


In [8]:
test

array([27086, 31792,   748, ...,   411,   369, 47496])

In [9]:
from transformers import GPT2LMHeadModel, AdamW

model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model_init.to(device);

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [10]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    1061 MB |    1061 MB |    1061 MB |       0 B  |
|       from large pool |    1060 MB |    1060 MB |    1060 MB |       0 B  |
|       from small pool |       0 MB |       0 MB |       0 MB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |    1061 MB |    1061 MB |    1061 MB |       0 B  |
|       from large pool |    1060 MB |    1060 MB |    1060 MB |

In [11]:
batch_size = 8
max_len = 256
epochs = 4

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

70 17


In [12]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                            #  attention_mask=batch_mask,
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print('epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/4 : training


  0%|          | 0/70 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/17 [00:00<?, ?it/s]

epoch 2/4 : training


...

  0%|          | 0/17 [00:00<?, ?it/s]

epoch 4/4 : training


  0%|          | 0/70 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/17 [00:00<?, ?it/s]

In [13]:
# https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate

import textwrap

# prompt = "print('Hello world!') "
prompt = 'Я СВАРЩИК И Я РАБОТАЮ БЕЗ ПЕРЕКУРРРААА'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=50,
    top_p=0.1,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Я СВАРЩИК И Я РАБОТАЮ БЕЗ ПЕРЕКУРРРАААААА!!!!!! Я РАБАТЫРЬ, РАБОТЯГА  Какое у меня будущее в этом году, когда я встречу
мужчину своей мечты, которого люблю, но он меня не любит, а я его люблю, он мне не нравится? У тебя есть парень, с
которым тебе будет интересно, а если нет, то ты его потеряешь.  Что делать, когда не хочется идти на свидание? Пойти на
свидание с парнем и сказать ему что ты его не любишь и что тебе не нравится. Попытайся найти себе другого. Если не
найдешь - не стоит и начинать.
------------------
Я СВАРЩИК И Я РАБОТАЮ БЕЗ ПЕРЕКУРРРАААААА!!!  И ЭТО НЕ МОЯ ЖЕНА, А МОЯ МАТЬ!!!  Я НЕ МОГУ СДЕЛАТЬ ЭТО, Я ПРОЩАЮСЬ!!!   Я
СТАЛ БОМЖЕЙ, Я УЖАСАЮСЯ, ЧТО МНЕ НЕ НУЖНА ТАКОЙ МАЛЕНЬКОЙ МАШИНЫ, КАК МАТЬ!  МАТЬ МЕНЯ НЕ ЛЮБИТ!  И МНЕ НАДО ВЫХОДИТЬ ИЗ
СВОЕГО СЕРДЦА!!!  А МАТЬ МОЯ ЛЮБИМАЯ, Я СМОГ УМЕРЛА!!!
------------------
Я СВАРЩИК И Я РАБОТАЮ БЕЗ ПЕРЕКУРРРАААААА!!!  Я СВОЕГО ВАС ЛЮБЛЮ!  С ВАШЕЙ ВАС ПОЗВОЛЯЮ!!!  СЛАВА ВАМ ВСЕМ!  И ВАМ
ОГРОМНАЯ ПОЛНОСТЬЮ ПОМОЩЬ!  Я ВСЕГО ЖДЁТ!!!   - Я

In [ ]:
import textwrap

prompt = 'Взял в руки топор и вкусное свежее мороженное'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=50,
    top_p=0.1,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Взял в руки топор и вкусное свежее мороженное. Он выпил залпом стакан, потом другой и опять выпил залпом, но уже без
всякого аппетита, так как не хотел, чтобы ему опять подали. «Ну, вот и все! Теперь уж не до еды! — думал он, — теперь
уже всё кончено, теперь уже я сам всё это придумал!» Он встал и вышел на улицу. Он шел и шел и думал, что теперь всё
кончено и всё кончено. Он не знал, как теперь жить и как жить дальше. Он был в отчаянии. Ему хотелось плакать и
смеяться, и ему хотелось смеяться и плакать. Он пошел домой и сел в угол, на диване. Ему было так хорошо и хорошо, как
никогда
------------------
Взял в руки топор и вкусное свежее мороженное. Он не хотел, чтобы кто-то видел его в таком виде, но все-таки пошел к
дверям, но, проходя, вдруг остановился и посмотрел в сторону. — Да, это ты! Он остановился, как вкопанный, как бы в
изумлении. «Что это он?» «Это ты! — мелькнуло у него в голове. — Это ты! Я тебя видел, я видел тебя, я тебя знаю! Ты
пришел ко мне! Я знаю, что ты здесь!» Но

In [ ]:
a = np.arange(27).reshape((3,3,3))
b = np.arange(27).reshape((3,9))
print(a)
print(b)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]]
[[ 0  1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16 17]
 [18 19 20 21 22 23 24 25 26]]


In [ ]:
def generate(prompt, len_gen=20, temperature=1):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model(context, past_key_values=past).values()
        # token = torch.argmax(output[..., -1, :], dim=-1)
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    return sequence

In [ ]:
import textwrap


prompt = """<!DOCTYPE html>
<html>
<body>

<h1>My First Heading</h1>
<p>My first paragraph.</p>"""
print(textwrap.fill(generate(prompt, 200, temperature=1), 120))


<!DOCTYPE html> <html> <body>  <h1>My First Heading</h1> <p>My first paragraph.</p> <p>My last chars.</p> <p>My first
tbody </p> <p>My last location</p> <p>My last location</p> <p>My exit</p> <p>My last description</p> <p>My first
position</p> <p>My longest description</p> <p>My last address</p>  </div>    <div class=row pamp <body>  <span class=row
pamp> <ul><span class=row pamp> <li> <span class=row pamp> <li> <span class=row pamp> <li> <span class=row pamp> <li> <
